In [1]:
import os
%pwd

'c:\\Trenches\\Text-Summarizer\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Trenches\\Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str 
    ALL_REQUIRED_FILES: list

In [5]:
from txtSummarizer.constants import *
from txtSummarizer.utils.common import read_yaml, create_directory

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directory([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            ALL_REQUIRED_FILES= config.ALL_REQUIRED_FILES,
        )

        return data_validation_config

In [7]:
import os
from txtSummarizer.logging import logging

In [8]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_files_exist(self) -> bool:
        """Validate that all files exist in the data directory"""
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    print(f"File {file} not found in data directory")
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    print(f"File {file} found in data directory")
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            print(e)
            raise e

    

In [9]:
#pipeline
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    print("Error in data validation: {0}".format(e))
    raise e

[2023-08-17 16:58:15,235: INFO: common: Config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/ladiyusuph/Text-Summarizer/raw/main/samsum_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}}]
[2023-08-17 16:58:15,241: INFO: common: Config: {'key': 'value'}]
[2023-08-17 16:58:15,245: INFO: common: Directory already exists: artifacts]
[2023-08-17 16:58:15,249: INFO: common: Created directory: artifacts/data_validation]
File dataset_dict.json not found in data directory
File test found in data directory
File train found in data directory
File validation found in data directory
